In [ ]:
# !pip install biopython

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   -------------- ------------------------- 1.0/2.8 MB 8.5 MB/s eta 0:00:01
   -------------------------- ------------- 1.8/2.8 MB 3.5 MB/s eta 0:00:01
   ----------------------------- ---------- 2.1/2.8 MB 3.8 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 3.2 MB/s eta 0:00:00


In [2]:
!pip install hmmlearn

   ---------------------------------------- 0.0/127.3 kB ? eta -:--:--
   --- ------------------------------------ 10.2/127.3 kB ? eta -:--:--
   --------- ----------------------------- 30.7/127.3 kB 435.7 kB/s eta 0:00:01
   -------------------------------------- - 122.9/127.3 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 127.3/127.3 kB 1.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from hmmlearn import hmm

In [2]:
def read_fasta(file_path, filter_header=None):
    sequences = ''
    for record in SeqIO.parse(file_path, "fasta"):
        if filter_header and filter_header not in record.id:
            continue
        sequences = record.seq
    return sequences

def read_centro_telo(file_path_centro, file_path_telo, file_path_cpg):
    centro = pd.read_csv(file_path_centro)
    centro.drop('#"bin"', axis=1, inplace=True)
    
    telo = pd.read_csv(file_path_telo)
    telo.drop('#bin', axis=1, inplace=True)
    telo = telo[telo['type'] == 'telomere']
    
    cpg = pd.read_csv(file_path_cpg)
    cpg.drop('#"bin"', axis=1, inplace=True)
    
    return centro, telo, cpg

def check_overlap(centro, telo, cpg):
    telo.sort_values('chromStart', inplace=True)
    # print("Starting Telomere")
    # print(f'({telo[['chromStart', 'chromEnd']].iloc[0, 0]},{telo[['chromStart', 'chromEnd']].iloc[0, 1]})')
    # print("Ending Telomere")
    # print(f'({telo[['chromStart', 'chromEnd']].iloc[1, 0]},{telo[['chromStart', 'chromEnd']].iloc[1, 1]})')
    
    start_telo = telo[['chromStart', 'chromEnd']].iloc[0, 1]
    end_telo = telo[['chromStart', 'chromEnd']].iloc[1, 0]

    if len(cpg[cpg['chromStart'] < start_telo]) > 0 or len(cpg[cpg['chromEnd'] > end_telo]) > 0:
        print("There is a CpG island overlapping with Telomeric regions.")
    else:
        print("There is NO overlapping CpG island with Telomeric regions.")
        
    centro.sort_values('chromStart', inplace=True)
    min_centro = centro[['chromStart','chromEnd']].iloc[0,0]
    max_centro = centro[['chromStart','chromEnd']].iloc[len(centro)-1, 1]

    if len(cpg[(cpg['chromStart'] > min_centro) & (cpg['chromEnd'] < max_centro)]) > 0:
        print("Do a manual inspection because there is a CpG island in between the Centromeric regions.")
    else:
        print("There is NO overlapping CpG island with Centromeric regions.")

In [5]:
seq = read_fasta('./GRCh38.p14.genome.fa', 'chr18')

In [3]:
centro18_df, telo18_df, cpg18_df = read_centro_telo('./chr18/centromeres.csv','./chr18/telomeres.csv', './chr18/cpg.csv')
centro19_df, telo19_df, cpg19_df = read_centro_telo('./chr19/centromeres.csv','./chr19/telomeres.csv', './chr19/cpg.csv')

In [7]:
check_overlap(centro18_df, telo18_df, cpg18_df)

There is NO overlapping CpG island with Telomeric regions.
There is NO overlapping CpG island with Centromeric regions.


In [5]:
check_overlap(centro19_df, telo19_df, cpg19_df)

There is NO overlapping CpG island with Telomeric regions.
There is NO overlapping CpG island with Centromeric regions.


In [6]:
def check_in_start_end(i, index, start_end, in_):
    if index < len(start_end):
        if not in_ and i >= start_end[index]:
            return True, index + 1
        if in_ and i >= start_end[index]:
            return False, index + 1
    return in_, index    

In [10]:
# check regions for ith base
in_cpg = False
cpg_index = 0

in_centro = False
centro_index = 0

in_telo = False
telo_index = 0

# check regions for (i+1)th base
in_cpg_next = False
cpg_index_next = 0

in_centro_next = False
centro_index_next = 0

in_telo_next = False
telo_index_next = 0

# flatten the regions
telo_start_end = telo18_df.sort_values('chromStart')[['chromStart', 'chromEnd']].to_numpy().flatten()
centro_start_end = centro18_df.sort_values('chromStart')[['chromStart', 'chromEnd']].to_numpy().flatten()
cpg_start_end = cpg18_df.sort_values('chromStart')[['chromStart', 'chromEnd']].to_numpy().flatten()

# Format: A-, G-, C-, T-, A+, G+, C+, T+
init_probs = np.zeros(8) # data structure for initial probabilities
emit_probs = np.concatenate([np.eye(4), np.eye(4)]) # data structure for emission probabilities
tran_probs = np.zeros((8,8)) # data structure for transition probabilities

# base to index
init_dict = {
    'A-': 0,
    'G-': 1,
    'C-': 2,
    'T-': 3,
    'A+': 4,
    'G+': 5,
    'C+': 6,
    'T+': 7,
}

# linearly traverse the sequence
for i in range(len(seq)):
    # check if in telomeric region
    in_telo, telo_index = check_in_start_end(i, telo_index, telo_start_end, in_telo)
    if in_telo:
        continue

    # check if in centromeric region
    in_centro, centro_index = check_in_start_end(i, centro_index, centro_start_end, in_centro)
    if in_centro:
        continue

    # check if in CpG island
    in_cpg, cpg_index = check_in_start_end(i, cpg_index, cpg_start_end, in_cpg)

    # avoid ambiguous bases N
    if seq[i] == 'N':
        continue

    # counting the frequency of each Markov States
    base_i = seq[i] + ('+' if in_cpg else '-')
    init_probs[init_dict[base_i]] += 1

    if i + 1 < len(seq):
        # check if next index in telomeric region
        in_telo_next, telo_index_next = check_in_start_end(i + 1, telo_index_next, telo_start_end, in_telo_next)
        if in_telo_next:
            continue

        # check if next base in centromeric region
        in_centro_next, centro_index_next = check_in_start_end(i + 1, centro_index_next, centro_start_end, in_centro_next)
        if in_centro_next:
            continue

        # check if next base in CpG island
        in_cpg_next, cpg_index_next = check_in_start_end(i + 1, cpg_index_next, cpg_start_end, in_cpg_next)

        # avoid ambiguous bases N
        if seq[i+1] == 'N':
            continue

        base_i_next = seq[i + 1] + ('+' if in_cpg_next else '-')
        tran_probs[init_dict[base_i_next], init_dict[base_i]] += 1

In [11]:
# sanity check for initial probabilities
(init_probs/init_probs.sum()).sum()

1.0

In [12]:
# sanity check for transition probabilities
(tran_probs/tran_probs.sum(axis=1)[:, np.newaxis]).sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1.])

In [13]:
final_tran_probs = tran_probs/tran_probs.sum(axis=1)[:, np.newaxis]
final_tran_probs

array([[3.36673990e-01, 1.95908309e-01, 2.39228107e-01, 2.28186162e-01,
        0.00000000e+00, 3.43276126e-06, 0.00000000e+00, 0.00000000e+00],
       [3.47248782e-01, 2.44139710e-01, 4.30627657e-02, 3.65536322e-01,
        0.00000000e+00, 1.24203267e-05, 0.00000000e+00, 0.00000000e+00],
       [2.54927379e-01, 2.02469356e-01, 2.44062515e-01, 2.98531777e-01,
        0.00000000e+00, 8.97388884e-06, 0.00000000e+00, 0.00000000e+00],
       [2.67780742e-01, 1.67633343e-01, 2.27956537e-01, 3.36624168e-01,
        0.00000000e+00, 5.21063363e-06, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.99077698e-01, 3.56594885e-01, 3.58744075e-01, 8.55833416e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.89002316e-01, 3.56557190e-01, 2.92117022e-01, 1.62323473e-01],
       [6.34074381e-04, 8.41119077e-04, 1.30696964e-03, 5.11141593e-04,
        1.24278579e-01, 3.47272186e-01, 3.57223272e-01, 1.

In [14]:
final_init_probs = init_probs/init_probs.sum()
final_init_probs

array([0.29997011, 0.19703748, 0.19670899, 0.30027983, 0.00093958,
       0.00205016, 0.00206689, 0.00094697])

In [15]:
# Checkpoint for transition probabilities
np.savetxt("transition_probs.csv", final_tran_probs, delimiter=",")

In [16]:
# Checkpoint forr initial probabilities
np.savetxt("initial_probs.csv", final_init_probs, delimiter=",")

### Build HMM

In [4]:
states = ['A-', 'G-', 'C-', 'T-', 'A+', 'G+', 'C+', 'T+']
n_states = len(states)

In [5]:
transition_probs = np.loadtxt("transition_probs.csv", delimiter=",")
initial_probs = np.loadtxt("initial_probs.csv", delimiter=",")

In [6]:
emit_probs = np.concatenate([np.eye(4), np.eye(4)])


In [7]:
transition_probs

array([[3.36673990e-01, 1.95908309e-01, 2.39228107e-01, 2.28186162e-01,
        0.00000000e+00, 3.43276126e-06, 0.00000000e+00, 0.00000000e+00],
       [3.47248782e-01, 2.44139710e-01, 4.30627657e-02, 3.65536322e-01,
        0.00000000e+00, 1.24203267e-05, 0.00000000e+00, 0.00000000e+00],
       [2.54927379e-01, 2.02469356e-01, 2.44062515e-01, 2.98531777e-01,
        0.00000000e+00, 8.97388884e-06, 0.00000000e+00, 0.00000000e+00],
       [2.67780742e-01, 1.67633343e-01, 2.27956537e-01, 3.36624168e-01,
        0.00000000e+00, 5.21063363e-06, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.99077698e-01, 3.56594885e-01, 3.58744075e-01, 8.55833416e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.89002316e-01, 3.56557190e-01, 2.92117022e-01, 1.62323473e-01],
       [6.34074381e-04, 8.41119077e-04, 1.30696964e-03, 5.11141593e-04,
        1.24278579e-01, 3.47272186e-01, 3.57223272e-01, 1.

In [8]:
model = hmm.CategoricalHMM(n_components=n_states)
model.startprob_= initial_probs
model.emissionprob_ = emit_probs
model.transmat_ = transition_probs

In [9]:
def get_gene_sequence(filename):
    file = read_fasta(filename)
    gene = str(file)
            
    out = []
    
    for char in gene :
        if char == "A":
            out.append([0])
        elif char == "G":
            out.append([1])
        elif char == "C":
            out.append([2])
        else:
            out.append([3])
            
    return np.array(out)
            


In [68]:
test_sequence_np = get_gene_sequence("seq_test_1_withoutN.fasta")

In [ ]:
logprob, output_bases = model.decode(test_sequence_np, algorithm="viterbi")

In [ ]:
out = "%s"%" ".join([states[x][-1] for x in output_bases])

In [78]:
with open ("output.txt", "w") as f:
    f.write(out)

In [11]:
test_sequence_np_2 = get_gene_sequence("seq_test_2_withoutN.fasta")

In [12]:
logprob_2, output_bases_2 = model.decode(test_sequence_np_2, algorithm="viterbi")

In [13]:
print(output_bases_2)

[1 0 3 ... 1 3 3]


In [14]:
np.unique(np.array(output_bases_2), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int64),
 array([5730616, 3881159, 3850131, 5744266, 2502474, 3569940, 3575744,
        2512412], dtype=int64))

In [15]:
out_2 = "%s"%" ".join([states[x][-1] for x in output_bases_2])

In [16]:
with open ("output_2.txt", "w") as f:
    f.write(out_2)

### Extract regions of chr19 WITHOUT telomere and centromere

In [12]:
seq_test = read_fasta('./GRCh38.p14.genome.fa', 'chr19')

In [13]:
check_overlap(centro19_df, telo19_df, cpg19_df)

There is NO overlapping CpG island with Telomeric regions.
There is NO overlapping CpG island with Centromeric regions.


In [46]:
# Extract 2 subsequences from chr19 such that
# 10000-first centromere
# skip all centromere regions 
# end of last centromere - (end- 10000)

start_centro = centro19_df['chromStart'][0]
end_centro = centro19_df['chromEnd'].iloc[-1]

end_first_telo = telo19_df['chromEnd'].iloc[0]
start_last_telo = telo19_df['chromStart'].iloc[-1]

seq_test_1 = seq_test[end_first_telo: start_centro]
seq_test_2 = seq_test[end_centro: start_last_telo]

# save the sequences to fasta files
with open('seq_test_1.fasta', 'w') as f:
    f.write(f'>seq_test_1\tindex {end_first_telo}:{start_centro-1}\n{seq_test_1}\n')

with open('seq_test_2.fasta', 'w') as f:
    f.write(f'>seq_test_2\tindex {end_centro}:{start_last_telo-1}\n{seq_test_2}\n')


In [48]:
# search for first index without N
def find_first_valid_index(seq):
    for i, base in enumerate(seq):
        if base != 'N':
            return i
    return -1

# search for first index with N
def find_first_invalid_index(seq):
    for i, base in enumerate(seq):
        if base == 'N':
            return i
    return -1

def find_valid_subsequence(seq):
    start_index = find_first_valid_index(seq)
    if start_index == -1:
        return 0, 0, ''

    end_index = find_first_invalid_index(seq[start_index:])
    if end_index == -1:
        return start_index, len(seq), seq[start_index:]

    end_index += start_index
    return start_index, end_index, seq[start_index:end_index]


start_index_1, end_index_1, valid_subsequence_1 = find_valid_subsequence(seq_test_1)
print (valid_subsequence_1[:100])
start_index_2, end_index_2, valid_subsequence_2 = find_valid_subsequence(seq_test_2)
print (valid_subsequence_2[:100])

with open('seq_test_1_withoutN.fasta', 'w') as f:
    f.write(f'>seq_test_1_withoutN\tindex {end_first_telo + start_index_1}:{end_first_telo+ end_index_1}\n{valid_subsequence_1}\n')
with open('seq_test_2_withoutN.fasta', 'w') as f:
    f.write(f'>seq_test_2_withoutN\tindex {end_centro + start_index_2}:{end_centro+ end_index_2}\n{valid_subsequence_2}\n')



GATCACAGAGGCTGGGCTGCTCCCCACCCTCTGCACACCTCCTGCTTCTAACAGCAGAGCTGCCAGGCCAGGCCCTCAGGCAAGGGCTCTGAAGTCAGGG
GATCCTTTACACAGAACAGATTTGAAACACTCTTTTTGTGGAATTTCCAAGTGGAGATTTCAGCCGATTTGAAGTCAATCGTAGAAAAGGAGATATCTTC


In [ ]:
cpg19_df.tail(20)

,chrom,chromStart,chromEnd,name,length,cpgNum,gcNum,perCpg,perGc,obsExp
0,chr19,70343,71344,CpG: 112,1001,112,731,22.4,73.0,0.86
1,chr19,267085,267660,CpG: 70,575,70,372,24.3,64.7,1.17
2,chr19,290594,292249,CpG: 165,1655,165,1028,19.9,62.1,1.04
3,chr19,295114,295354,CpG: 25,240,25,175,20.8,72.9,0.79
4,chr19,306214,306681,CpG: 40,467,40,339,17.1,72.6,0.66
...,...,...,...,...,...,...,...,...,...,...
2487,chr19,58558566,58559733,CpG: 115,1167,115,769,19.7,65.9,0.91
2488,chr19,58562046,58563388,CpG: 147,1342,147,933,21.9,69.5,0.91
2489,chr19,58572834,58573692,CpG: 80,858,80,520,18.6,60.6,1.04
2490,chr19,58575330,58575932,CpG: 60,602,60,411,19.9,68.3,0.86
